In [1]:
import json
import numpy as np
import os
import pandas as pd
import time

cwd = os.getcwd()
pkg_dir = os.path.dirname(cwd)
data_dir = os.path.join(pkg_dir, 'tests/data')

from reV.handlers.transmission import TransmissionFeatures as TF
from reV.supply_curve.supply_curve import SupplyCurve
from reV.utilities.loggers import setup_logger

solar_fcr = 0.096
wind_fcr = 0.079


In [2]:
path = os.path.join(data_dir, 'trans_tables', 'ri_transmission_table.csv')
trans_table = pd.read_csv(path)

path = os.path.join(data_dir, 'sc_out', 'baseline_agg_summary.csv')
sc_points = pd.read_csv(path)

path = os.path.join(data_dir, 'trans_tables', 'transmission_multipliers.csv')
multipliers = pd.read_csv(path)

In [3]:
logger = setup_logger('reV.supply_curve')
sc = SupplyCurve(sc_points, trans_table, fcr=0.096, sc_features=multipliers)
connections = sc.full_sort()
for h in logger.handlers:
    logger.removeHandler(h)

INFO - 2019-09-26 11:51:33,579 [supply_curve.py:365] : 1.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,580 [supply_curve.py:365] : 2.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,581 [supply_curve.py:365] : 3.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,582 [supply_curve.py:365] : 4.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,583 [supply_curve.py:365] : 5.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,585 [supply_curve.py:365] : 6.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,588 [supply_curve.py:365] : 7.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,588 [supply_curve.py:365] : 8.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,590 [supply_curve.py:365] : 9.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,591 [supply_curve.py:365] : 10.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,592 [supply_curve.py:365] : 11.0

INFO - 2019-09-26 11:51:33,796 [supply_curve.py:365] : 88.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,798 [supply_curve.py:365] : 89.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,799 [supply_curve.py:365] : 90.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,801 [supply_curve.py:365] : 91.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,802 [supply_curve.py:365] : 92.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,803 [supply_curve.py:365] : 93.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,805 [supply_curve.py:365] : 94.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,807 [supply_curve.py:365] : 95.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,809 [supply_curve.py:365] : 96.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,810 [supply_curve.py:365] : 97.0 % of supply curve points connected
INFO - 2019-09-26 11:51:33,812 [supply_curve.py:36

In [4]:
# Full sort
ts = time.time()
sc_full = SupplyCurve.full(sc_points, trans_table, fcr=0.096, sc_features=multipliers)
print('Full Supply Curve = {:.4f} seconds'.format(time.time() - ts))

# Full 4 workers
ts = time.time()
sc_full_p = SupplyCurve.full(sc_points, trans_table, fcr=0.096, sc_features=multipliers, max_workers=4)
print('Full Supply Curve w/ 4 workers = {:.4f} seconds'.format(time.time() - ts))

# Simple
ts = time.time()
sc_simple = SupplyCurve.simple(sc_points, trans_table, fcr=0.096, sc_features=multipliers)
print('Simple Supply Curve {:.4f} seconds'.format(time.time() - ts))


# SC v1
ts = time.time()
sc_vintage = SupplyCurve.full(sc_points, trans_table, fcr=0.096, line_limited=True, sc_features=multipliers)
print('Vintage Supply Curve = {:.4f} seconds'.format(time.time() - ts))

Full Supply Curve = 1.4810 seconds
Full Supply Curve w/ 4 workers = 1.4080 seconds
Simple Supply Curve 1.2423 seconds
Vintage Supply Curve = 1.4103 seconds


In [5]:
sc_full.equals(sc_full_p)

True

In [5]:
sc_vintage.head()

,area_sq_km,capacity,gen_gids,gid_counts,latitude,longitude,mean_cf,mean_lcoe,mean_res,pct_slope,...,sc_point_gid,sc_row_ind,sc_gid,transmission_multiplier,trans_gid,trans_type,lcot,total_lcoe,dist_mi,trans_cap_cost
0,2.3409,84.2724,[7],[289],41.993,-71.807999,0.124,124.002823,3.991000,3.195715,...,1,0,0,2.25,43636,TransLine,7.95737,131.96,9.215849,90037.664062
1,1.8954,68.2344,[7],[234],41.993,-71.737999,0.124,124.002823,3.991000,3.121439,...,2,0,1,2.25,43451,TransLine,1.58331,125.586,0.474527,17915.203125
2,8.0595,290.1420,"[22, 40]","[719, 276]",41.993,-71.737999,0.124,123.848488,4.005335,2.845650,...,2,0,2,2.25,80843,PCALoadCen,16.0604,139.909,22.025019,181722.921875
3,4.8438,174.3768,[66],[598],41.993,-71.668999,0.124,124.039299,3.996000,2.908158,...,3,0,3,2.25,80843,PCALoadCen,14.3803,138.42,19.720990,162712.968750
4,3.3696,121.3056,"[40, 76]","[356, 60]",41.993,-71.668999,0.124,123.855751,4.003163,2.859689,...,3,0,4,2.25,68810,Substation,0.85944,124.715,1.178627,9724.557617
